In [ ]:
import numpy as np
import pandas as pd

In [ ]:
def dv_importFromRaw(filepath, T=False):

    try:
        with open(filepath, 'rb') as f:
            RAW = f.read
            data = np.frombuffer(RAW(),
                                 dtype=np.ushort,
                                 offset=0).reshape(1280, 768, 768)
            if T:
                values = data.T
            else:
                values = data

            return values

    except IOError:
        print("Error while opening the file!")

scan = dv_importFromRaw(".\\training\\volumes\scan_001.raw", T=True)

In [ ]:
import pyvista as pv

data = pv.wrap(scan)
data.plot(volume=True) # Volume render

In [ ]:
xyz.flatten()

In [58]:
#xy = [[xy, xy] for xy in range(0, 768*0.49479, 0.49479)]

xyz= []
for z in np.arange(0, 1280*0.3125, 0.3125):
    for xy in np.arange(0, 768*0.49479, 0.49479):
        xyz.append([xy, xy, z])
xyz = np.array(xyz)

test = np.array([[x, y, z] for x in np.arange(0, 768*0.49479, 0.49479) for y in np.arange(0, 768*0.49479, 0.49479) for z in np.arange(0, 1280*0.3125, 0.3125)])

In [ ]:
import open3d as o3d

pcd = o3d.geometry.PointCloud() # create an empty poun cloud object

pcd.points = o3d.utility.Vector3dVector(xyz) # feed the xyz coordinates

#pcd.colors = o3d.utility.Vector3dVector(scan.flat) # feed the rgb colors

#pcd.normals = o3d.utility.Vector3dVector(fmi_xyz_n) # feed the normal vectors

o3d.visualization.draw_geometries([pcd], point_show_normal=True) # display the pcd

In [ ]:
grid = pv.UniformGrid(dimensions=(1280, 768, 768), spacing=(0.49479, 0.49479, 0.3125), origin=(0, 0, 0), data)

In [ ]:
test = np.load('fmi_xyz.npy')